<a href="https://colab.research.google.com/github/Rtarachan02/ResearchWork/blob/main/Notebook_AdaptiveMCT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Datasets()

For converting TXT file to CSV file.

## .txt to .csv 512 x16  or other dims check well

In [ ]:
path='/content/drive/My Drive/FOG AND EDGE/Adaptive MCT/1024x32/A.u_c_hihi'
with open(path) as f:
    lines = f.readlines()
lines=lines[1:]
data=[float(x.strip()) for x in lines]
# data=lines
uslolo=[data[i:i+32] for i in range(0, len(data), 32)]
uslolodf=pd.DataFrame(uslolo)
print(uslolodf.shape)
uslolodf.tail(5)
path2='/content/drive/My Drive/FOG AND EDGE/Adaptive MCT/1024x32/A.u_c_hihi.csv'
uslolodf.to_csv(path2,index=False)

(1024, 32)


## preprocess data

string formatis converted to floating point, if any space is there it will be checked.

In [ ]:
path='/content/drive/My Drive/FOG AND EDGE/Assignment2/u_s_hihi.csv'
data = pd.read_csv(path)
data=data[:512]
data.drop(data.columns[0], axis=1, inplace=True)
data.head(25)
# data=data[:,1:]
data.to_csv(path, index=False)

## Append All (Optional for merged instances case)

In [ ]:
import pandas as pd

# List of file paths
csv_paths = [
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_c_lolo.csv',
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_c_hihi.csv',
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_c_hilo.csv',
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_c_lohi.csv',
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_i_lolo.csv',
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_i_hihi.csv',
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_i_hilo.csv',
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_i_lohi.csv',
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_s_lolo.csv',
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_s_hihi.csv',
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_s_hilo.csv',
    '/content/drive/My Drive/FOG AND EDGE/Assignment2/u_s_lohi.csv'
]

# Read and concatenate all the files in the list
merged_df = pd.concat([pd.read_csv(path) for path in csv_paths], ignore_index=True)

# Check the last few rows of the merged DataFrame
# print(merged_df.tail())
# merged_df.tail(25)
merged_df=merged_df.drop(merged_df.columns[-1], axis=1)
merged_df.head(25)
# Save the merged DataFrame to a new CSV file
merged_df.to_csv('/content/drive/My Drive/FOG AND EDGE/Assignment2/tasks_machines.csv', index=False)


# CNN Model

In [ ]:
!ls 'drive/My Drive/Sanjaya Kumar Panda'

'Ongoing Papers'


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam

class CNNModel:
    def __init__(self, input_shape, filters=32, kernel_size=(3, 3), loss='mse'):
        self.input_shape = input_shape
        self.filters = filters
        self.kernel_size = kernel_size
        self.loss = loss
        self.model = self._build_model()

    def _build_model(self, learning_rate=0.001):
        input_layer = Input(shape=self.input_shape)
        conv = Conv2D(filters=self.filters, kernel_size=self.kernel_size, padding='same', activation='relu')(input_layer)
        conv = Conv2D(filters=self.filters, kernel_size=self.kernel_size, padding='same', activation='relu')(conv)
        flatten = Flatten()(conv)
        output_layer = Dense(16)(flatten)

        # Create Adam optimizer with a placeholder learning rate
        self.optimizer = Adam(learning_rate=learning_rate)

        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer=self.optimizer, loss=self.loss)
        return model

    def train(self, X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, learning_rate=0.005):
        # Update the learning rate dynamically before training
        self.optimizer.learning_rate = learning_rate

        # Rebuild the model to apply the new learning rate
        self.model = self._build_model(learning_rate=learning_rate)

        history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        return history

    def predict(self, X_test):
        y_pred = self.model.predict(X_test)
        return y_pred

    @staticmethod
    def create_sliding_windows(data, window_size=10):
        X, y = [], []
        for i in range(len(data) - window_size):
            X.append(data[i:i+window_size])
            y.append(data[i+window_size])
        return np.array(X), np.array(y)


In [ ]:
%%capture
# path='/content/drive/My Drive/FOG AND EDGE/Adaptive MCT/DataGen/data2.csv'
path='/content/drive/My Drive/Sanjaya Kumar Panda/Ongoing Papers/Tara/Adaptive MCT/u_c_hihi.csv'
# path='/content/drive/My Drive/Adaptive MCT/u_c_hihi.csv'
data_df = pd.read_csv(path)
data=data_df.to_numpy()
data = np.array(data, dtype=str)
data = np.char.strip(data)  #
data[data == ''] = '0.0'
data=data.astype(float)
cnn_model_instance = CNNModel(input_shape=(10, 16,1))
X, y = CNNModel.create_sliding_windows(data)
cnn_model_instance.train(X, y, epochs=500, batch_size=32, validation_split=0.2)
X_test = X  # Replace with actual test data
# y_pred = cnn_model_instance.predict(X_test)

KeyboardInterrupt: 

In [ ]:
X_test = X  # Replace with actual test data
y_pred = cnn_model_instance.predict(X_test)
print(y_pred[0])

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
[  11809.727  304252.25   524484.94   904485.56   961816.2   1122293.5
 1149944.2   1589249.1   1883334.1   1892437.6   1936501.4   1973629.9
 2250294.8   2409231.8   2470604.2   2687712.2  ]


# ConvLSTM Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, ConvLSTM2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam

class ConvLSTMModel:
    def __init__(self, input_shape, filters=32, kernel_size=(3, 1), loss='mse'):
        self.input_shape = input_shape
        self.filters = filters
        self.kernel_size = kernel_size
        self.loss = loss
        self.model = self._build_model()

    def _build_model(self, learning_rate=0.001):
        input_layer = Input(shape=self.input_shape)
        # Using ConvLSTM2D for capturing spatial and temporal information
        conv_lstm = ConvLSTM2D(filters=self.filters, kernel_size=self.kernel_size, padding='same', activation='relu', return_sequences=True)(input_layer)
        conv_lstm = ConvLSTM2D(filters=self.filters, kernel_size=self.kernel_size, padding='same', activation='relu')(conv_lstm)
        flatten = Flatten()(conv_lstm)
        output_layer = Dense(16)(flatten)

        # Create Adam optimizer with a placeholder learning rate
        self.optimizer = Adam(learning_rate=learning_rate)

        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer=self.optimizer, loss=self.loss)
        return model

    def train(self, X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, learning_rate=0.005):
        # Update the learning rate dynamically before training
        self.optimizer.learning_rate = learning_rate

        # Rebuild the model to apply the new learning rate
        self.model = self._build_model(learning_rate=learning_rate)

        history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        return history

    def predict(self, X_test):
        y_pred = self.model.predict(X_test)
        return y_pred

    @staticmethod
    def create_sliding_windows(data, window_size=10):
        X, y = [], []
        for i in range(len(data) - window_size):
            X.append(data[i:i+window_size])
            y.append(data[i+window_size])
        return np.array(X), np.array(y)


In [ ]:
%%capture
# path='/content/drive/My Drive/FOG AND EDGE/Adaptive MCT/DataGen/data2.csv'
path='/content/drive/My Drive/FOG AND EDGE/Adaptive MCT/u_c_hihi.csv'
data_df = pd.read_csv(path)
data=data_df.to_numpy()
data = np.array(data, dtype=str)
data = np.char.strip(data)  #
data[data == ''] = '0.0'
data=data.astype(float)
convlstm_model_instance = ConvLSTMModel(input_shape=(10, 16,1,1))
X, y = ConvLSTMModel.create_sliding_windows(data)
X = X.reshape(X.size//160, 10, 16, 1, 1)
convlstm_model_instance.train(X, y, epochs=200, batch_size=32, validation_split=0.2)
X_test = X  # Replace with actual test data
# y_pred = convlstm_model_instance.predict(X_test)


In [ ]:
X_test = X  # Replace with actual test data
y_pred = convlstm_model_instance.predict(X_test)
print(y_pred[0])

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step
[  93531.06  173828.33  255149.14  356616.2   436332.72  509989.28
  567891.44  659520.44  743520.2   804809.3   897672.44  974077.94
 1065892.4  1145219.5  1221060.5  1307839.6 ]


# TCNN Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Dense, Flatten
from tensorflow.keras.optimizers import Adam

class TCNNModel:
    def __init__(self, input_shape, filters=32, kernel_size=2, loss='mse'):
        self.input_shape = input_shape
        self.filters = filters
        self.kernel_size = kernel_size
        self.loss = loss
        self.model = self._build_model()

    def _build_model(self, learning_rate=0.001):
        input_layer = Input(shape=self.input_shape)
        conv = Conv1D(filters=self.filters, kernel_size=self.kernel_size, padding='causal', activation='relu')(input_layer)
        conv = Conv1D(filters=self.filters, kernel_size=self.kernel_size, padding='causal', activation='relu')(conv)
        flatten = Flatten()(conv)
        output_layer = Dense(16)(flatten)

        # Create Adam optimizer with a placeholder learning rate
        self.optimizer = Adam(learning_rate=learning_rate)

        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer=self.optimizer, loss=self.loss)
        return model

    def train(self, X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, learning_rate=0.005):
        # Update the learning rate dynamically before training
        self.optimizer.learning_rate = learning_rate

        # Rebuild the model to apply the new learning rate
        self.model = self._build_model(learning_rate=learning_rate)

        history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        return history

    def predict(self, X_test):
        y_pred = self.model.predict(X_test)
        return y_pred

    @staticmethod
    def create_sliding_windows(data, window_size=10):
        X, y = [], []
        for i in range(len(data) - window_size):
            X.append(data[i:i+window_size])
            y.append(data[i+window_size])
        return np.array(X), np.array(y)

In [ ]:
%%capture
# path='/content/drive/My Drive/FOG AND EDGE/Adaptive MCT/DataGen/data2.csv'
path='/content/drive/My Drive/FOG AND EDGE/Adaptive MCT/u_c_hihi.csv'
data_df = pd.read_csv(path)
data=data_df.to_numpy()
data = np.array(data, dtype=str)
data = np.char.strip(data)  #
data[data == ''] = '0.0'
data=data.astype(float)
tcnn_model_instance = TCNNModel(input_shape=(10, 16))
X, y = TCNNModel.create_sliding_windows(data)
tcnn_model_instance.train(X, y, epochs=500, batch_size=32, validation_split=0.2)
X_test = X  # Replace with actual test data
# y_pred = tcnn_model_instance.predict(X_test)


In [ ]:
X_test = X  # Replace with actual test data
y_pred = tcnn_model_instance.predict(X_test)
print(y_pred[0])

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
[  99049.87  304976.25  536345.9   847528.94  940096.3  1114247.9
 1235481.5  1613519.6  1896066.8  2004983.4  2110446.5  2231784.
 2520562.5  2618435.5  2707569.   2903516.2 ]


## save model

In [ ]:

# Save the model in Keras format
model_instance.model.save('/content/drive/My Drive/FOG AND EDGE/Assignment2/tcnn_model.keras')


# Benchmark MCT Algorithm

In [ ]:
import numpy as np

class TaskScheduler:
    def __init__(self, num_resources, tasks):
        self.num_resources = num_resources
        self.tasks = np.array(tasks)  # Only execution times for simplicity
        self.resources = np.zeros(num_resources)  # Initialize resources' completion times

    def assign_tasks(self):
        task_assignments = []
        # for task_execution_times in self.tasks:
        for i in range(len(self.tasks)):
            task_execution_times = self.tasks[i]
            # Find the resource with the minimum current completion time
            min_completion_time_resource = np.argmin(self.resources + task_execution_times)
            # Assign the task to this resource
            self.resources[min_completion_time_resource] += task_execution_times[min_completion_time_resource]
            task_assignments.append((i, min_completion_time_resource))

        return task_assignments

    def calculate_makespan(self):
        # The makespan is the maximum value in resources array
        return np.max(self.resources)



In [ ]:
# path2='/content/drive/MyDrive/FOG AND EDGE/Adaptive MCT/DataGen/data2.csv'
path2='/content/drive/MyDrive/FOG AND EDGE/Adaptive MCT/1024x32/A.u_c_hihi.csv'
data_df = pd.read_csv(path2)
data=data_df.to_numpy()
data = np.array(data, dtype=str)
data = np.char.strip(data)  #
# data[data == ''] = '260561'
tasks=data.astype(float)
num_resources = 32
scheduler = TaskScheduler(num_resources=num_resources, tasks=tasks)
task_assignments = scheduler.assign_tasks()
makespan = scheduler.calculate_makespan()
print("Makespan using Benchmark MCT is",makespan)

Makespan using Benchmark MCT is 32832741.92


# Adaptive MCT Alogorithms

In [ ]:
class TaskScheduler:
    def __init__(self, num_resources, tasks, model_instance,model_code):
        self.num_resources = num_resources
        self.model_code=model_code
        self.tasks = np.array(tasks)  # Only execution times
        self.model_instance = model_instance
        self.resources = np.zeros(num_resources)  # Initialize resources' completion times
    def predict_execution_times(self, task_data):
        # task_data = task_data[np.newaxis, ...]
        return self.model_instance.predict(task_data)

    def calculate_flf(self, predicted_execution_times):
        # FLF is directly the predicted execution times for simplicity
        return predicted_execution_times

    def calculate_wf(self, exec_time, min_exec_time, max_exec_time, task_size):
        return ((exec_time - min_exec_time) / (max_exec_time - min_exec_time)) * (1 - np.exp(-task_size))

    def assign_tasks(self):
        task_assignments = []
        for i in range(len(self.tasks)):
            execution_time = self.tasks[i]
            min_exec_time = np.min(execution_time)
            max_exec_time = np.max(execution_time)

            if i >= 10:
              if self.model_code==1:#CNN
                # Prepare input data for prediction: use the last 10 tasks
                input_data = self.tasks[i-10:i].reshape(1, 10,16)  # Reshape to 1 sample with 10 features
                predicted_execution_time = self.predict_execution_times(input_data)
              elif self.model_code==2:#ConvLSTM
                # Prepare input data for prediction: use the last
                input_data = self.tasks[i-10:i].reshape(1, 10,16,1,1)  # Reshape to 1 sample with 10 features
                predicted_execution_time = self.predict_execution_times(input_data)
              elif self.model_code==3:#TCNN
                # Prepare input data for prediction: use the last 10 tasks
                input_data = self.tasks[i-10:i].reshape(1, 10,16)  # Reshape to 1 sample with 10 features
                predicted_execution_time = self.predict_execution_times(input_data)
            else:
                predicted_execution_time = np.zeros(16)

            ready_time = np.min(self.resources)  # Determine the ready_time based on resource completion times
            flf = self.calculate_flf(predicted_execution_time)
            wf = self.calculate_wf(execution_time, min_exec_time, max_exec_time, task_size=512)

            # Compute adaptive values for all resources
            adaptive_values = ready_time + execution_time + (flf * wf)

            # Compute the adaptive value for each resource
            resource_adaptive_values = self.resources + adaptive_values

            # Find resource with the least adaptive value
            assigned_resource = np.argmin(resource_adaptive_values)
            self.resources[assigned_resource] += execution_time[assigned_resource]  # Update resource's completion time

            task_assignments.append((i, assigned_resource, adaptive_values))  # Include task index

        return task_assignments

    def calculate_makespan(self):
        return np.max(self.resources)


# Assign Tasks  : Run for each instance One by one

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# Load TCNN model
%%capture
# model_path = '/content/drive/My Drive/FOG AND EDGE/Assignment2/tcnn_model.keras'  # Replace with your actual model path
# tc_model = load_model(model_path)
# model_instance=model_instance
# path='/content/drive/My Drive/FOG AND EDGE/Assignment2/u_c_hihi.csv'
# path2='/content/drive/MyDrive/FOG AND EDGE/Adaptive MCT/DataGen/data2.csv'
path2='/content/drive/MyDrive/FOG AND EDGE/Adaptive MCT/u_c_hihi.csv'
data_df = pd.read_csv(path2)
data=data_df.to_numpy()
data = np.array(data, dtype=str)
data = np.char.strip(data)  #
# data[data == ''] = '260561'
tasks=data.astype(float)
num_resources = 16
#CNN
model_instance=cnn_model_instance
scheduler = TaskScheduler(num_resources=num_resources, tasks=tasks, model_instance=model_instance,model_code=1)
cnn_task_assignments = scheduler.assign_tasks()
cnn_makespan = scheduler.calculate_makespan()


#ConvLSTM
model_instance=convlstm_model_instance
scheduler = TaskScheduler(num_resources=num_resources, tasks=tasks, model_instance=model_instance,model_code=2)
convlstm_task_assignments = scheduler.assign_tasks()
convlstm_makespan = scheduler.calculate_makespan()


#TCNN
model_instance=tcnn_model_instance
scheduler = TaskScheduler(num_resources=num_resources, tasks=tasks, model_instance=model_instance,model_code=3)
tcnn_task_assignments = scheduler.assign_tasks()
tcnn_makespan = scheduler.calculate_makespan()


In [ ]:
#Benchmark MCT
print("Task Assignments:", [list(s)[:2] for s in task_assignments])
print("Makespan using Benchmark MCT is",makespan)
print("Task Assignments:", [list(s)[:2] for s in cnn_task_assignments])
print("Makespan using Adaptive MCT with CNN is", cnn_makespan)
print("Task Assignments:", [list(s)[:2] for s in convlstm_task_assignments])
print("Makespan using Adaptive MCT with ConvLSTM is", convlstm_makespan)
print("Task Assignments:", [list(s)[:2] for s in tcnn_task_assignments])
print("Makespan using Adaptive MCT with TCNN is", tcnn_makespan)


Task Assignments: [[0, 0], [1, 1], [2, 0], [3, 1], [4, 2], [5, 3], [6, 0], [7, 1], [8, 0], [9, 4], [10, 0], [11, 5], [12, 4], [13, 6], [14, 7], [15, 0], [16, 2], [17, 1], [18, 8], [19, 5], [20, 0], [21, 6], [22, 0], [23, 3], [24, 2], [25, 9], [26, 0], [27, 7], [28, 9], [29, 10], [30, 8], [31, 1], [32, 0], [33, 3], [34, 2], [35, 1], [36, 0], [37, 11], [38, 12], [39, 1], [40, 4], [41, 13], [42, 5], [43, 3], [44, 6], [45, 14], [46, 4], [47, 12], [48, 7], [49, 0], [50, 12], [51, 15], [52, 10], [53, 14], [54, 15], [55, 2], [56, 11], [57, 8], [58, 0], [59, 2], [60, 9], [61, 11], [62, 1], [63, 15], [64, 4], [65, 2], [66, 3], [67, 11], [68, 3], [69, 0], [70, 1], [71, 8], [72, 15], [73, 0], [74, 5], [75, 5], [76, 2], [77, 15], [78, 10], [79, 15], [80, 0], [81, 0], [82, 7], [83, 13], [84, 4], [85, 14], [86, 1], [87, 2], [88, 6], [89, 1], [90, 5], [91, 3], [92, 15], [93, 0], [94, 12], [95, 4], [96, 2], [97, 11], [98, 13], [99, 7], [100, 2], [101, 12], [102, 0], [103, 8], [104, 9], [105, 1], [106,

In [ ]:
import pandas as pd

# Assuming task IDs are available in the range of task length
num_tasks = len(task_assignments)  # Get the number of tasks

# Create a DataFrame to store the task assignments and makespan for each method
result = pd.DataFrame(columns=['TaskId', 'Benchmark_MCT', 'CNN_Adaptive_MCT', 'ConvLSTM_Adaptive_MCT', 'TCNN_Adaptive_MCT'])

# Create a list to collect rows before concatenating
rows = []

# Append the task assignments and corresponding method results into the list of rows
for i in range(num_tasks):
    rows.append({
        'TaskId': i,
        'Benchmark_MCT': task_assignments[i][1],  # Resource for Benchmark MCT
        'CNN_Adaptive_MCT': cnn_task_assignments[i][1],  # Resource for CNN Adaptive MCT
        'ConvLSTM_Adaptive_MCT': convlstm_task_assignments[i][1],  # Resource for ConvLSTM Adaptive MCT
        'TCNN_Adaptive_MCT': tcnn_task_assignments[i][1]  # Resource for TCNN Adaptive MCT
    })

# Concatenate the list of rows into the DataFrame
result = pd.concat([result, pd.DataFrame(rows)], ignore_index=True)

# Print the makespan for each method
print("Makespan using Benchmark MCT is", makespan)
print("Makespan using Adaptive MCT with CNN is", cnn_makespan)
print("Makespan using Adaptive MCT with ConvLSTM is", convlstm_makespan)
print("Makespan using Adaptive MCT with TCNN is", tcnn_makespan)

# Display the DataFrame
rpath='/content/drive/MyDrive/FOG AND EDGE/Adaptive MCT/Results and Reports/Result_UCHIHI1.csv'
result.to_csv(rpath, index=False)
result.tail()


Makespan using Benchmark MCT is 11422624.494138002
Makespan using Adaptive MCT with CNN is 12138201.110101005
Makespan using Adaptive MCT with ConvLSTM is 11526548.37158
Makespan using Adaptive MCT with TCNN is 11911431.562495999


,TaskId,Benchmark_MCT,CNN_Adaptive_MCT,ConvLSTM_Adaptive_MCT,TCNN_Adaptive_MCT
507,507,2,0,1,2
508,508,1,0,0,0
509,509,7,7,7,1
510,510,10,8,9,10
511,511,3,6,3,4
